# **Modelling Pipeline**

The intended use of this .ipynb file is to take the processed data file and train multiple models aimed at detecting pedestrians, all while employing Mlflow tracking.

## Download imports needed

In [ ]:
import sys
import os
import getpass

import mlflow
import mlflow.pytorch

import torch
import utils
import torchvision

from plotly.subplots import make_subplots
import plotly.graph_objects as go

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

from codecarbon import EmissionsTracker
from google.colab import drive

from data import load_data
from segmentation_map import draw_segmentation_map
from modelling import evaluation_mflow, train_mlflow

## Configure Emissions Tracker

In [ ]:
! pip install codecarbon

[codecarbon INFO @ 08:51:18] Energy consumed for RAM : 0.002339 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 08:51:18] Energy consumed for all GPUs : 0.023357 kWh. Total GPU Power : 31.12172412665487 W
[codecarbon INFO @ 08:51:18] Energy consumed for all CPUs : 0.020922 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 08:51:18] 0.046618 kWh of electricity used since the beginning.


In [ ]:
tracker = EmissionsTracker(
    output_file = 'emissions.csv',
    on_csv_write = 'append',
)
tracker.start()

[codecarbon INFO @ 08:51:19] [setup] RAM Tracking...
[codecarbon INFO @ 08:51:19] [setup] GPU Tracking...
[codecarbon INFO @ 08:51:19] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 08:51:19] [setup] CPU Tracking...
[codecarbon WARNING @ 08:51:19] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 08:51:20] We saw that you have a Intel(R) Xeon(R) CPU @ 2.30GHz but we don't know it. Please contact us.
[codecarbon INFO @ 08:51:20] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.30GHz
[codecarbon INFO @ 08:51:20] >>> Tracker's metadata:
[codecarbon INFO @ 08:51:20]   Platform system: Linux-5.15.120+-x86_64-with-glibc2.35
[codecarbon INFO @ 08:51:20]   Python version: 3.10.12
[codecarbon INFO @ 08:51:20]   CodeCarbon version: 2.3.1
[codecarbon INFO @ 08:51:20]   Available RAM : 12.678 GB
[codecarbon INFO @ 08:51:20]   CPU count: 2
[codecarbon INFO @ 08:51:20]   CPU model: Intel(R) Xeon(R) CPU @ 2.30GHz
[codecarbon INFO @ 08:51:20]   GPU

## Load the processed data

In [ ]:
# We retrieve the data from the google drive folder
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
%%shell

# TorchVision repo is downloaded to use some reference files
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.15.1

cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

fatal: destination path 'vision' already exists and is not an empty directory.
HEAD is now at 42759b1cc8 Version number bump for vision (#7419)


In [ ]:
# Add the directory to the current Python path

# Replace the path with the correct path for each individual (the path has been set up to work for all three members of the team)
DATA_FOLDER = "/content/drive/MyDrive/TAED2-PedestrianDetection/Datasets/PennFudan/Processed data"
sys.path.append(DATA_FOLDER)

FUNCTIONS_FOLDER = "/content/drive/MyDrive/TAED2-PedestrianDetection/Milestone 2"
sys.path.append(FUNCTIONS_FOLDER)


In [ ]:
# Load each dataset from their respective file.
# Download preprocessed data

training_dataset, validation_dataset, testing_dataset = load_data(DATA_FOLDER)

## Set up Mlflow

In [ ]:
!pip install mlflow

[codecarbon INFO @ 08:51:33] Energy consumed for RAM : 0.002359 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 08:51:33] Energy consumed for all GPUs : 0.023487 kWh. Total GPU Power : 31.233257212161686 W
[codecarbon INFO @ 08:51:33] Energy consumed for all CPUs : 0.021099 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 08:51:33] 0.046945 kWh of electricity used since the beginning.


In [ ]:
# set environment variables
password = getpass.getpass(prompt='Enter your password: ')

os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/RodrigoBonferroni/taed2-PedestrianDetection.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "claudiamur"
os.environ["MLFLOW_TRACKING_PASSWORD"] = password

mlflow.set_tracking_uri(os.environ["MLFLOW_TRACKING_URI"])

## Training the models

In [ ]:

# define the data loaders

BATCH_SIZE_TRAIN = 2

data_loader = torch.utils.data.DataLoader(
    training_dataset, batch_size=BATCH_SIZE_TRAIN, shuffle=True, num_workers=2,
    collate_fn=utils.collate_fn)

data_loader_val = torch.utils.data.DataLoader(
    validation_dataset, batch_size=1, shuffle=False, num_workers=2,
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    testing_dataset, batch_size=1, shuffle=False, num_workers=2,
    collate_fn=utils.collate_fn)

### Pre-trained Mask R-CNN model

In [ ]:
# Import pre-trained model mask rcnn resnet 50

model = torchvision.models.detection.maskrcnn_resnet50_fpn_v2(weights='DEFAULT', progress=True)

[codecarbon INFO @ 08:51:35] Energy consumed for RAM : 0.000020 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 08:51:35] Energy consumed for all GPUs : 0.000130 kWh. Total GPU Power : 31.23234291426314 W
[codecarbon INFO @ 08:51:35] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 08:51:35] 0.000327 kWh of electricity used since the beginning.


Set the model GPU and evaluation mode:

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {device}.")
model.to(device).eval() # we will evaluate the pre-trained model

Using cuda.


MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
         

Let us show the prediction of the pre-trained model, as a reminder we use this input image with the following groundtruth:

In [ ]:
(img,target) = validation_dataset[24]

fig = make_subplots(rows=1, cols=2, subplot_titles=("Input", "Ground Truth"))
fig.add_trace(go.Image(z=img.numpy().transpose(1,2,0)*255), 1, 1)
fig.add_trace(go.Image(z=draw_segmentation_map(img, target)*255), 1, 2)

Output hidden; open in https://colab.research.google.com to view.

Pass the image (as a batch) through the model:

In [ ]:
# add a batch dimension
imgs = img.unsqueeze(0).to(device) #torch.stack((img,img))
outs = model(imgs.to(device))

Let's check how many predictions, labels and scores are found for this image (see the COCO_NAMES list for the correspondence between label numbers and semantic meaning):

In [ ]:
print(f"Number of predictions = {len(outs[0]['labels'])}")
print(f"  labels = {outs[0]['labels'].cpu().numpy()}")
print(f"  scores = {outs[0]['scores'].detach().cpu().numpy()}")

Number of predictions = 30
  labels = [ 1  1  1 40  1 31 31 31 31 34 15 27 27 27 62 27 15 27  1 64 64 27  3 27
 40 40  1 40 62 77]
  scores = [0.99934536 0.9943652  0.9049078  0.8923801  0.85306126 0.8071016
 0.6772084  0.533892   0.46627563 0.19908205 0.19770248 0.19313224
 0.14463395 0.14262018 0.12126844 0.1084424  0.09878358 0.09870253
 0.09803258 0.09207785 0.09154422 0.07993338 0.06456899 0.0641988
 0.06177687 0.05430169 0.05390199 0.05319972 0.05013453 0.05002739]


Let's show the results:

In [ ]:
fig = make_subplots(rows=1, cols=2, subplot_titles=("Prediction (all scores)", "Prediction (scores>0.8)"))
fig.add_trace(go.Image(z=draw_segmentation_map(img, outs[0], score_thres=0.0)*255), 1, 1)
fig.add_trace(go.Image(z=draw_segmentation_map(img, outs[0], score_thres=0.8)*255), 1, 2)

Output hidden; open in https://colab.research.google.com to view.

#### Testing the pre-trained Mask R-CNN model

We will evaluate the model:

Now let's defined a general function for tracking evaluation metrics.

In [ ]:
# Evaluate the pre-trained Mask R-CNN model

tracker.stop()
tracker.start()
mlflow.set_experiment("Sense fine tunning")
evaluation_mflow('eval', data_loader_val, model, device)
tracker.stop()

creating index...
index created!
Test:  [ 0/50]  eta: 0:00:44  model_time: 0.3582 (0.3582)  evaluator_time: 0.0609 (0.0609)  time: 0.8954  data: 0.4749  max mem: 4945


[codecarbon INFO @ 08:51:48] Energy consumed for RAM : 0.002378 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 08:51:48] Energy consumed for all GPUs : 0.023644 kWh. Total GPU Power : 37.509823500254306 W
[codecarbon INFO @ 08:51:48] Energy consumed for all CPUs : 0.021276 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 08:51:48] 0.047298 kWh of electricity used since the beginning.
[codecarbon INFO @ 08:51:50] Energy consumed for RAM : 0.000040 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 08:51:50] Energy consumed for all GPUs : 0.000304 kWh. Total GPU Power : 41.70123032906837 W
[codecarbon INFO @ 08:51:50] Energy consumed for all CPUs : 0.000355 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 08:51:50] 0.000698 kWh of electricity used since the beginning.


Test:  [49/50]  eta: 0:00:00  model_time: 0.1610 (0.2047)  evaluator_time: 0.0213 (0.0486)  time: 0.2082  data: 0.0071  max mem: 4945
Test: Total time: 0:00:14 (0.2805 s / it)
Averaged stats: model_time: 0.1610 (0.2047)  evaluator_time: 0.0213 (0.0486)
Accumulating evaluation results...
DONE (t=0.03s).
Accumulating evaluation results...
DONE (t=0.03s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.857
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.984
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.939
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.359
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.647
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.884
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.419
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 1

In [ ]:
torch.save(model, 'pretrained.pth')

## Train Mask-RCNN on our dataset - Fine tunning

We can change the classifier of the Mask-RCNN to train it with this dataset.

In [ ]:
NUM_CLASSES = 2
# load an instance segmentation model pre-trained on COCO
model2 = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True, progress=True)

# get the number of input features for the classifier
in_features = model2.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model2.roi_heads.box_predictor = FastRCNNPredictor(in_features, NUM_CLASSES)

# now get the number of input features for the mask classifier
in_features_mask = model2.roi_heads.mask_predictor.conv5_mask.in_channels
HIDDEN_LAYER = 256
# and replace the mask predictor with a new one
model2.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                    HIDDEN_LAYER,
                                                    NUM_CLASSES)

model2.to(device)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning:

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning:

Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MaskRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.



MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(in

In [ ]:
# Fine tunning experiments
mlflow.set_experiment("Fine tunning")

<Experiment: artifact_location='mlflow-artifacts:/8fe5dacbb5f94df0a76166f7b8b5feec', creation_time=1696181498306, experiment_id='3', last_update_time=1696181498306, lifecycle_stage='active', name='Fine tunning', tags={}>

#### Baseline

In [ ]:
tracker.start()
train_mlflow(model2, data_loader, data_loader_val, num_epochs=3, hidden_layer=HIDDEN_LAYER,
    batch_size_train=BATCH_SIZE_TRAIN, name="baseline", device=device)
tracker.stop()

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning:

The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning:

The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tens

Epoch: [0]  [  0/140]  eta: 0:02:18  lr: 0.000041  loss: 3.7912 (3.7912)  loss_classifier: 0.5827 (0.5827)  loss_box_reg: 0.1133 (0.1133)  loss_mask: 3.0810 (3.0810)  loss_objectness: 0.0121 (0.0121)  loss_rpn_box_reg: 0.0020 (0.0020)  time: 0.9912  data: 0.3921  max mem: 4945


[codecarbon INFO @ 08:52:03] Energy consumed for RAM : 0.002399 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 08:52:03] Energy consumed for all GPUs : 0.023883 kWh. Total GPU Power : 56.64613136744091 W
[codecarbon INFO @ 08:52:03] Energy consumed for all CPUs : 0.021456 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 08:52:03] 0.047737 kWh of electricity used since the beginning.
[codecarbon INFO @ 08:52:05] Energy consumed for RAM : 0.000060 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 08:52:05] Energy consumed for all GPUs : 0.000551 kWh. Total GPU Power : 58.81340470189025 W
[codecarbon INFO @ 08:52:05] Energy consumed for all CPUs : 0.000533 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 08:52:05] 0.001144 kWh of electricity used since the beginning.


Epoch: [0]  [ 10/140]  eta: 0:01:23  lr: 0.000400  loss: 1.8172 (2.3849)  loss_classifier: 0.4595 (0.4418)  loss_box_reg: 0.3305 (0.3058)  loss_mask: 1.1690 (1.6060)  loss_objectness: 0.0238 (0.0261)  loss_rpn_box_reg: 0.0039 (0.0051)  time: 0.6456  data: 0.0452  max mem: 5006
Epoch: [0]  [ 20/140]  eta: 0:01:15  lr: 0.000760  loss: 1.1950 (1.7057)  loss_classifier: 0.2662 (0.3329)  loss_box_reg: 0.3305 (0.3111)  loss_mask: 0.5749 (1.0306)  loss_objectness: 0.0238 (0.0250)  loss_rpn_box_reg: 0.0059 (0.0061)  time: 0.6068  data: 0.0112  max mem: 5006


[codecarbon INFO @ 08:52:18] Energy consumed for RAM : 0.002418 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 08:52:18] Energy consumed for all GPUs : 0.024173 kWh. Total GPU Power : 69.64259293638155 W
[codecarbon INFO @ 08:52:18] Energy consumed for all CPUs : 0.021633 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 08:52:18] 0.048224 kWh of electricity used since the beginning.


...

[codecarbon INFO @ 08:59:05] Energy consumed for RAM : 0.002954 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 08:59:05] Energy consumed for all GPUs : 0.031500 kWh. Total GPU Power : 68.49760845509991 W
[codecarbon INFO @ 08:59:05] Energy consumed for all CPUs : 0.026430 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 08:59:05] 0.060885 kWh of electricity used since the beginning.


Test:  [139/140]  eta: 0:00:00  model_time: 0.2273 (0.2371)  evaluator_time: 0.0090 (0.0136)  time: 0.2559  data: 0.0085  max mem: 5467
Test: Total time: 0:00:37 (0.2693 s / it)
Averaged stats: model_time: 0.2273 (0.2371)  evaluator_time: 0.0090 (0.0136)
Accumulating evaluation results...
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.01s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.233
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.257
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.257
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.204
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.235
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.083
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets

[codecarbon INFO @ 08:59:07] Energy consumed for RAM : 0.000616 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 08:59:07] Energy consumed for all GPUs : 0.008154 kWh. Total GPU Power : 64.5984092738195 W
[codecarbon INFO @ 08:59:07] Energy consumed for all CPUs : 0.005510 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 08:59:07] 0.014280 kWh of electricity used since the beginning.


creating index...
index created!
Test:  [ 0/50]  eta: 0:00:22  model_time: 0.1635 (0.1635)  evaluator_time: 0.0061 (0.0061)  time: 0.4515  data: 0.2792  max mem: 5467
Test:  [49/50]  eta: 0:00:00  model_time: 0.0986 (0.1049)  evaluator_time: 0.0027 (0.0042)  time: 0.1123  data: 0.0043  max mem: 5467
Test: Total time: 0:00:06 (0.1227 s / it)
Averaged stats: model_time: 0.0986 (0.1049)  evaluator_time: 0.0027 (0.0042)
Accumulating evaluation results...
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.02s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.847
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.989
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.971
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.401
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.709
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | max

[codecarbon INFO @ 08:59:20] Energy consumed for RAM : 0.002974 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 08:59:20] Energy consumed for all GPUs : 0.031702 kWh. Total GPU Power : 48.50786113049746 W
[codecarbon INFO @ 08:59:20] Energy consumed for all CPUs : 0.026607 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 08:59:20] 0.061283 kWh of electricity used since the beginning.
2023/10/16 08:59:21 WARNING mlflow.utils.requirements_utils: Found torch version (2.0.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.0.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
[codecarbon INFO @ 08:59:22] Energy consumed for RAM : 0.000636 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 08:59:22] Energy consumed for all GPUs : 0.008349 kWh. Total GPU Power : 46.858860210140605 W
[codecarbon I

In [ ]:
torch.save(model2, 'baseline.pth')

And we can compare the test results with the pre-trained model. As we have a small dataset with not much variability, the small training helps achieving slightly better results.

In [ ]:
(img,target) = validation_dataset[22]
imgs = img.unsqueeze(0).to(device) #torch.stack((img,img))
model2.eval()
outs = model2(imgs)

fig = make_subplots(rows=1, cols=2, subplot_titles=("Prediction (all scores)", "Prediction (scores>0.8)"))
fig.add_trace(go.Image(z=draw_segmentation_map(img, outs[0], score_thres=0.0)*255), 1, 1)
fig.add_trace(go.Image(z=draw_segmentation_map(img, outs[0], score_thres=0.8)*255), 1, 2)

Output hidden; open in https://colab.research.google.com to view.

#### Change hidden_layer

In [ ]:
# now get the number of input features for the mask classifier
in_features_mask = model2.roi_heads.mask_predictor.conv5_mask.in_channels
HIDDEN_LAYER = 128
# and replace the mask predictor with a new one
model2.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                    HIDDEN_LAYER,
                                                    NUM_CLASSES)

model2.to(device)


MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(in

In [ ]:
tracker.start()
train_mlflow(model2, data_loader, data_loader_val, num_epochs=3, hidden_layer=HIDDEN_LAYER,
    batch_size_train=BATCH_SIZE_TRAIN, name="hidden128", device=device)
tracker.stop()

[codecarbon INFO @ 08:59:37] Energy consumed for RAM : 0.000655 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 08:59:37] Energy consumed for all GPUs : 0.008480 kWh. Total GPU Power : 31.441055260735567 W
[codecarbon INFO @ 08:59:37] Energy consumed for all CPUs : 0.005864 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 08:59:37] 0.015000 kWh of electricity used since the beginning.


Epoch: [0]  [  0/140]  eta: 0:03:20  lr: 0.000041  loss: 1.7799 (1.7799)  loss_classifier: 0.0191 (0.0191)  loss_box_reg: 0.0265 (0.0265)  loss_mask: 1.7294 (1.7294)  loss_objectness: 0.0002 (0.0002)  loss_rpn_box_reg: 0.0046 (0.0046)  time: 1.4343  data: 0.7384  max mem: 5467
Epoch: [0]  [ 10/140]  eta: 0:01:27  lr: 0.000400  loss: 1.3400 (1.2974)  loss_classifier: 0.0195 (0.0203)  loss_box_reg: 0.0265 (0.0313)  loss_mask: 1.2540 (1.2429)  loss_objectness: 0.0002 (0.0004)  loss_rpn_box_reg: 0.0020 (0.0025)  time: 0.6744  data: 0.0850  max mem: 5467


[codecarbon INFO @ 08:59:50] Energy consumed for RAM : 0.003014 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 08:59:50] Energy consumed for all GPUs : 0.032090 kWh. Total GPU Power : 60.17525074747551 W
[codecarbon INFO @ 08:59:50] Energy consumed for all CPUs : 0.026964 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 08:59:50] 0.062068 kWh of electricity used since the beginning.


Epoch: [0]  [ 20/140]  eta: 0:01:14  lr: 0.000760  loss: 0.7642 (0.9411)  loss_classifier: 0.0195 (0.0195)  loss_box_reg: 0.0257 (0.0318)  loss_mask: 0.6594 (0.8868)  loss_objectness: 0.0003 (0.0004)  loss_rpn_box_reg: 0.0025 (0.0025)  time: 0.5765  data: 0.0156  max mem: 5467


[codecarbon INFO @ 08:59:52] Energy consumed for RAM : 0.000675 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 08:59:52] Energy consumed for all GPUs : 0.008758 kWh. Total GPU Power : 66.33276825511447 W
[codecarbon INFO @ 08:59:52] Energy consumed for all CPUs : 0.006043 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 08:59:52] 0.015476 kWh of electricity used since the beginning.


...

[codecarbon INFO @ 09:06:53] Energy consumed for RAM : 0.001230 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:06:53] Energy consumed for all GPUs : 0.016212 kWh. Total GPU Power : 66.5953569581249 W
[codecarbon INFO @ 09:06:53] Energy consumed for all CPUs : 0.011006 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:06:53] 0.028448 kWh of electricity used since the beginning.


creating index...
index created!
Test:  [ 0/50]  eta: 0:00:18  model_time: 0.1641 (0.1641)  evaluator_time: 0.0060 (0.0060)  time: 0.3610  data: 0.1896  max mem: 5467


[codecarbon INFO @ 09:07:07] Energy consumed for RAM : 0.003590 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:07:07] Energy consumed for all GPUs : 0.039766 kWh. Total GPU Power : 46.02049904926254 W
[codecarbon INFO @ 09:07:07] Energy consumed for all CPUs : 0.032118 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:07:07] 0.075474 kWh of electricity used since the beginning.


Test:  [49/50]  eta: 0:00:00  model_time: 0.0987 (0.1055)  evaluator_time: 0.0038 (0.0054)  time: 0.1199  data: 0.0079  max mem: 5467
Test: Total time: 0:00:06 (0.1262 s / it)
Averaged stats: model_time: 0.0987 (0.1055)  evaluator_time: 0.0038 (0.0054)
Accumulating evaluation results...
DONE (t=0.03s).
Accumulating evaluation results...
DONE (t=0.02s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.835
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.983
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.983
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.233
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.719
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.856
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.406
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 1

[codecarbon INFO @ 09:07:08] Energy consumed for RAM : 0.001250 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:07:08] Energy consumed for all GPUs : 0.016406 kWh. Total GPU Power : 46.5706714608032 W
[codecarbon INFO @ 09:07:08] Energy consumed for all CPUs : 0.011183 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:07:08] 0.028839 kWh of electricity used since the beginning.
2023/10/16 09:07:10 WARNING mlflow.utils.requirements_utils: Found torch version (2.0.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.0.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2023/10/16 09:07:14 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.15.2+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torchvision==0.15.2' without the 

In [ ]:
torch.save(model2, 'hidden128.pth')

In [ ]:
# now get the number of input features for the mask classifier
in_features_mask = model2.roi_heads.mask_predictor.conv5_mask.in_channels
HIDDEN_LAYER = 512
# and replace the mask predictor with a new one
model2.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                    HIDDEN_LAYER,
                                                    NUM_CLASSES)

model2.to(device)

MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(in

In [ ]:
tracker.start()
train_mlflow(model2, data_loader, data_loader_val, num_epochs=3, hidden_layer=HIDDEN_LAYER,
    batch_size_train=BATCH_SIZE_TRAIN, name="hidden512", device=device)
tracker.stop()

[codecarbon INFO @ 09:16:38] Energy consumed for RAM : 0.004344 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:16:38] Energy consumed for all GPUs : 0.048523 kWh. Total GPU Power : 35.90309142514469 W
[codecarbon INFO @ 09:16:38] Energy consumed for all CPUs : 0.038862 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:16:38] 0.091729 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:16:40] Energy consumed for RAM : 0.002004 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:16:40] Energy consumed for all GPUs : 0.025160 kWh. Total GPU Power : 32.911776398110455 W
[codecarbon INFO @ 09:16:40] Energy consumed for all CPUs : 0.017933 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:16:40] 0.045097 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:16:53] Energy consumed for RAM : 0.004363 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:16:53] Energy consumed for all GPUs : 0.048655 kWh. Total GPU Power : 31.5968730997

Epoch: [0]  [  0/140]  eta: 0:02:42  lr: 0.000041  loss: 0.1078 (0.1078)  loss_classifier: 0.0071 (0.0071)  loss_box_reg: 0.0101 (0.0101)  loss_mask: 0.0898 (0.0898)  loss_objectness: 0.0002 (0.0002)  loss_rpn_box_reg: 0.0007 (0.0007)  time: 1.1625  data: 0.5305  max mem: 5467
Epoch: [0]  [ 10/140]  eta: 0:01:27  lr: 0.000400  loss: 0.1078 (0.1117)  loss_classifier: 0.0093 (0.0091)  loss_box_reg: 0.0104 (0.0122)  loss_mask: 0.0880 (0.0884)  loss_objectness: 0.0001 (0.0001)  loss_rpn_box_reg: 0.0016 (0.0019)  time: 0.6755  data: 0.0566  max mem: 5467


[codecarbon INFO @ 09:17:53] Energy consumed for RAM : 0.004443 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:17:53] Energy consumed for all GPUs : 0.049266 kWh. Total GPU Power : 51.325031627111514 W
[codecarbon INFO @ 09:17:53] Energy consumed for all CPUs : 0.039751 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:17:53] 0.093459 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:17:55] Energy consumed for RAM : 0.002104 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:17:55] Energy consumed for all GPUs : 0.025918 kWh. Total GPU Power : 55.595774223414075 W
[codecarbon INFO @ 09:17:55] Energy consumed for all CPUs : 0.018821 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:17:55] 0.046842 kWh of electricity used since the beginning.


Epoch: [0]  [ 20/140]  eta: 0:01:14  lr: 0.000760  loss: 0.1157 (0.1174)  loss_classifier: 0.0093 (0.0096)  loss_box_reg: 0.0125 (0.0131)  loss_mask: 0.0907 (0.0929)  loss_objectness: 0.0001 (0.0002)  loss_rpn_box_reg: 0.0015 (0.0016)  time: 0.5895  data: 0.0116  max mem: 5467
Epoch: [0]  [ 30/140]  eta: 0:01:07  lr: 0.001119  loss: 0.1162 (0.1167)  loss_classifier: 0.0089 (0.0095)  loss_box_reg: 0.0108 (0.0125)  loss_mask: 0.0950 (0.0931)  loss_objectness: 0.0001 (0.0002)  loss_rpn_box_reg: 0.0011 (0.0015)  time: 0.5757  data: 0.0138  max mem: 5467


[codecarbon INFO @ 09:18:08] Energy consumed for RAM : 0.004463 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:18:08] Energy consumed for all GPUs : 0.049555 kWh. Total GPU Power : 69.55662560131154 W
[codecarbon INFO @ 09:18:08] Energy consumed for all CPUs : 0.039928 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:18:08] 0.093946 kWh of electricity used since the beginning.


...

[codecarbon INFO @ 09:24:40] Energy consumed for RAM : 0.004980 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:24:40] Energy consumed for all GPUs : 0.056641 kWh. Total GPU Power : 68.39078108484267 W
[codecarbon INFO @ 09:24:40] Energy consumed for all CPUs : 0.044550 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:24:40] 0.106171 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:24:42] Energy consumed for RAM : 0.002640 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:24:42] Energy consumed for all GPUs : 0.033297 kWh. Total GPU Power : 68.32214524929867 W
[codecarbon INFO @ 09:24:42] Energy consumed for all CPUs : 0.023623 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:24:42] 0.059560 kWh of electricity used since the beginning.


Test:  [139/140]  eta: 0:00:00  model_time: 0.2331 (0.2374)  evaluator_time: 0.0139 (0.0135)  time: 0.2626  data: 0.0113  max mem: 5467
Test: Total time: 0:00:37 (0.2668 s / it)
Averaged stats: model_time: 0.2331 (0.2374)  evaluator_time: 0.0139 (0.0135)
Accumulating evaluation results...
DONE (t=0.03s).
Accumulating evaluation results...
DONE (t=0.04s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.245
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.257
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.257
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.233
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.246
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.086
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets

[codecarbon INFO @ 09:24:55] Energy consumed for RAM : 0.004999 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:24:55] Energy consumed for all GPUs : 0.056872 kWh. Total GPU Power : 55.46344483356213 W
[codecarbon INFO @ 09:24:55] Energy consumed for all CPUs : 0.044727 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:24:55] 0.106599 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:24:57] Energy consumed for RAM : 0.002660 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:24:57] Energy consumed for all GPUs : 0.033528 kWh. Total GPU Power : 55.49421073177818 W
[codecarbon INFO @ 09:24:57] Energy consumed for all CPUs : 0.023800 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:24:57] 0.059988 kWh of electricity used since the beginning.


Test:  [49/50]  eta: 0:00:00  model_time: 0.0985 (0.1050)  evaluator_time: 0.0038 (0.0046)  time: 0.1158  data: 0.0054  max mem: 5467
Test: Total time: 0:00:06 (0.1241 s / it)
Averaged stats: model_time: 0.0985 (0.1050)  evaluator_time: 0.0038 (0.0046)
Accumulating evaluation results...
DONE (t=0.03s).
Accumulating evaluation results...
DONE (t=0.02s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.836
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.971
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.971
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.118
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.744
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.862
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.412
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 1

2023/10/16 09:24:59 WARNING mlflow.utils.requirements_utils: Found torch version (2.0.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.0.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2023/10/16 09:25:04 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.15.2+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torchvision==0.15.2' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
[codecarbon INFO @ 09:25:10] Energy consumed for RAM : 0.005019 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:25:10] Energy consumed for all GPUs : 0.057040 kWh. Total GPU Power : 40.20227037237507 W
[codecarbon INFO @ 0

In [ ]:
torch.save(model2, 'hidden512.pth')

In [ ]:
# Restart configuration

# now get the number of input features for the mask classifier
in_features_mask = model2.roi_heads.mask_predictor.conv5_mask.in_channels
HIDDEN_LAYER = 256
# and replace the mask predictor with a new one
model2.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                    HIDDEN_LAYER,
                                                    NUM_CLASSES)

model2.to(device)


MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(in

#### Change num_epochs

In [ ]:
NUM_EPOCHS = 2

In [ ]:
tracker.start()
train_mlflow(model2, data_loader, data_loader_val, num_epochs=NUM_EPOCHS, hidden_layer=HIDDEN_LAYER,
    batch_size_train=BATCH_SIZE_TRAIN, name="2epochs", device=device)
tracker.stop()

Epoch: [0]  [  0/140]  eta: 0:01:58  lr: 0.000041  loss: 0.1197 (0.1197)  loss_classifier: 0.0049 (0.0049)  loss_box_reg: 0.0092 (0.0092)  loss_mask: 0.1043 (0.1043)  loss_objectness: 0.0002 (0.0002)  loss_rpn_box_reg: 0.0010 (0.0010)  time: 0.8494  data: 0.2616  max mem: 5467


[codecarbon INFO @ 09:29:56] Energy consumed for RAM : 0.005397 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:29:56] Energy consumed for all GPUs : 0.060845 kWh. Total GPU Power : 47.02867478728717 W
[codecarbon INFO @ 09:29:56] Energy consumed for all CPUs : 0.048280 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:29:56] 0.114521 kWh of electricity used since the beginning.


Epoch: [0]  [ 10/140]  eta: 0:01:20  lr: 0.000400  loss: 0.1176 (0.1182)  loss_classifier: 0.0084 (0.0094)  loss_box_reg: 0.0117 (0.0130)  loss_mask: 0.0926 (0.0946)  loss_objectness: 0.0002 (0.0002)  loss_rpn_box_reg: 0.0008 (0.0009)  time: 0.6208  data: 0.0333  max mem: 5467


[codecarbon INFO @ 09:29:57] Energy consumed for RAM : 0.003056 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:29:57] Energy consumed for all GPUs : 0.037490 kWh. Total GPU Power : 50.42822388502418 W
[codecarbon INFO @ 09:29:57] Energy consumed for all CPUs : 0.027346 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:29:57] 0.067892 kWh of electricity used since the beginning.


Epoch: [0]  [ 20/140]  eta: 0:01:13  lr: 0.000760  loss: 0.1132 (0.1190)  loss_classifier: 0.0092 (0.0096)  loss_box_reg: 0.0117 (0.0132)  loss_mask: 0.0886 (0.0951)  loss_objectness: 0.0001 (0.0002)  loss_rpn_box_reg: 0.0008 (0.0010)  time: 0.6003  data: 0.0105  max mem: 5467
Epoch: [0]  [ 30/140]  eta: 0:01:07  lr: 0.001119  loss: 0.1124 (0.1209)  loss_classifier: 0.0099 (0.0099)  loss_box_reg: 0.0108 (0.0131)  loss_mask: 0.0886 (0.0968)  loss_objectness: 0.0001 (0.0002)  loss_rpn_box_reg: 0.0008 (0.0010)  time: 0.6062  data: 0.0105  max mem: 5467


...

[codecarbon INFO @ 09:33:58] Energy consumed for RAM : 0.005715 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:33:58] Energy consumed for all GPUs : 0.065142 kWh. Total GPU Power : 44.222439403831096 W
[codecarbon INFO @ 09:33:58] Energy consumed for all CPUs : 0.051135 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:33:58] 0.121992 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:33:58] Energy consumed for RAM : 0.003373 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:33:58] Energy consumed for all GPUs : 0.041764 kWh. Total GPU Power : 44.230623946925064 W
[codecarbon INFO @ 09:33:58] Energy consumed for all CPUs : 0.030186 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:33:58] 0.075323 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:34:13] Energy consumed for RAM : 0.005735 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:34:13] Energy consumed for all GPUs : 0.065427 kWh. Total GPU Power : 68.148035582

Test:  [100/140]  eta: 0:00:10  model_time: 0.2371 (0.2384)  evaluator_time: 0.0102 (0.0131)  time: 0.2601  data: 0.0092  max mem: 5467


[codecarbon INFO @ 09:34:28] Energy consumed for RAM : 0.005755 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:34:28] Energy consumed for all GPUs : 0.065713 kWh. Total GPU Power : 68.72191342121093 W
[codecarbon INFO @ 09:34:28] Energy consumed for all CPUs : 0.051489 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:34:28] 0.122957 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:34:28] Energy consumed for RAM : 0.003413 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:34:28] Energy consumed for all GPUs : 0.042334 kWh. Total GPU Power : 68.75389960020775 W
[codecarbon INFO @ 09:34:28] Energy consumed for all CPUs : 0.030541 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:34:28] 0.076288 kWh of electricity used since the beginning.


Test:  [139/140]  eta: 0:00:00  model_time: 0.2357 (0.2363)  evaluator_time: 0.0089 (0.0128)  time: 0.2566  data: 0.0084  max mem: 5467
Test: Total time: 0:00:37 (0.2668 s / it)
Averaged stats: model_time: 0.2357 (0.2363)  evaluator_time: 0.0089 (0.0128)
Accumulating evaluation results...
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.01s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.243
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.257
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.248
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.233
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.245
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.087
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets

2023/10/16 09:34:42 WARNING mlflow.utils.requirements_utils: Found torch version (2.0.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.0.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
[codecarbon INFO @ 09:34:43] Energy consumed for RAM : 0.005775 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:34:43] Energy consumed for all GPUs : 0.065949 kWh. Total GPU Power : 56.57080652756458 W
[codecarbon INFO @ 09:34:43] Energy consumed for all CPUs : 0.051666 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:34:43] 0.123390 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:34:43] Energy consumed for RAM : 0.003433 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:34:43] Energy consumed for all GPUs : 0.042570 kWh. Total GPU Power : 56.556581203805486 W
[codecarbon I

In [ ]:
torch.save(model2, '2epochs.pth')

#### Change batch_size

In [ ]:
BATCH_SIZE_TRAIN = 8

# redefine the data loader
data_loader = torch.utils.data.DataLoader(
    training_dataset, batch_size=BATCH_SIZE_TRAIN, shuffle=True, num_workers=2,
    collate_fn=utils.collate_fn)

In [ ]:
tracker.start()
train_mlflow(model2, data_loader, data_loader_val, num_epochs = 3, hidden_layer=HIDDEN_LAYER,
    batch_size_train=BATCH_SIZE_TRAIN, name="batch8", device=device)
tracker.stop()

[codecarbon INFO @ 09:36:14] Energy consumed for RAM : 0.003552 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:36:14] Energy consumed for RAM : 0.005894 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:36:14] Energy consumed for all GPUs : 0.043358 kWh. Total GPU Power : 36.20907425639185 W
[codecarbon INFO @ 09:36:14] Energy consumed for all GPUs : 0.066737 kWh. Total GPU Power : 36.12392661766474 W
[codecarbon INFO @ 09:36:14] Energy consumed for all CPUs : 0.052735 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:36:14] 0.125366 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:36:14] Energy consumed for all CPUs : 0.031786 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:36:14] 0.078696 kWh of electricity used since the beginning.


Epoch: [0]  [ 0/35]  eta: 0:02:28  lr: 0.000152  loss: 0.1241 (0.1241)  loss_classifier: 0.0124 (0.0124)  loss_box_reg: 0.0133 (0.0133)  loss_mask: 0.0971 (0.0971)  loss_objectness: 0.0002 (0.0002)  loss_rpn_box_reg: 0.0011 (0.0011)  time: 4.2503  data: 1.3272  max mem: 10867


[codecarbon INFO @ 09:36:29] Energy consumed for RAM : 0.003572 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:36:29] Energy consumed for all GPUs : 0.043644 kWh. Total GPU Power : 68.9827154979847 W
[codecarbon INFO @ 09:36:29] Energy consumed for all CPUs : 0.031962 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:36:29] 0.079179 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:36:29] Energy consumed for RAM : 0.005914 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:36:29] Energy consumed for all GPUs : 0.067023 kWh. Total GPU Power : 68.78287227276317 W
[codecarbon INFO @ 09:36:29] Energy consumed for all CPUs : 0.052912 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:36:29] 0.125849 kWh of electricity used since the beginning.


Epoch: [0]  [10/35]  eta: 0:01:15  lr: 0.001621  loss: 0.1105 (0.1138)  loss_classifier: 0.0102 (0.0099)  loss_box_reg: 0.0133 (0.0129)  loss_mask: 0.0913 (0.0899)  loss_objectness: 0.0001 (0.0002)  loss_rpn_box_reg: 0.0009 (0.0010)  time: 3.0047  data: 0.1530  max mem: 11027


[codecarbon INFO @ 09:36:44] Energy consumed for RAM : 0.003592 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:36:44] Energy consumed for all GPUs : 0.043933 kWh. Total GPU Power : 69.2451481482659 W
[codecarbon INFO @ 09:36:44] Energy consumed for all CPUs : 0.032139 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:36:44] 0.079664 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:36:44] Energy consumed for RAM : 0.005934 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:36:44] Energy consumed for all GPUs : 0.067311 kWh. Total GPU Power : 69.2621363030445 W
[codecarbon INFO @ 09:36:44] Energy consumed for all CPUs : 0.053089 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:36:44] 0.126334 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:36:59] Energy consumed for RAM : 0.003611 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:36:59] Energy consumed for all GPUs : 0.044221 kWh. Total GPU Power : 69.3867804500596

...

[codecarbon INFO @ 09:44:04] Energy consumed for RAM : 0.006514 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:44:04] Energy consumed for all GPUs : 0.075172 kWh. Total GPU Power : 63.9916250911781 W
[codecarbon INFO @ 09:44:04] Energy consumed for all CPUs : 0.058285 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:44:04] 0.139971 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:44:04] Energy consumed for RAM : 0.004172 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:44:04] Energy consumed for all GPUs : 0.051794 kWh. Total GPU Power : 63.968768792455265 W
[codecarbon INFO @ 09:44:04] Energy consumed for all CPUs : 0.037335 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:44:04] 0.093301 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:44:19] Energy consumed for RAM : 0.006534 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:44:19] Energy consumed for all GPUs : 0.075454 kWh. Total GPU Power : 67.52082789630

Test:  [34/35]  eta: 0:00:01  model_time: 1.1328 (1.1125)  evaluator_time: 0.0359 (0.0431)  time: 1.2243  data: 0.0326  max mem: 11997
Test: Total time: 0:00:42 (1.2181 s / it)
Averaged stats: model_time: 1.1328 (1.1125)  evaluator_time: 0.0359 (0.0431)
Accumulating evaluation results...
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.01s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.256
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.257
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.257
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.257
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.256
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.088
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=

[codecarbon INFO @ 09:44:34] Energy consumed for RAM : 0.006554 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:44:34] Energy consumed for all GPUs : 0.075723 kWh. Total GPU Power : 64.66442693251203 W
[codecarbon INFO @ 09:44:34] Energy consumed for all CPUs : 0.058639 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:44:34] 0.140916 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:44:34] Energy consumed for RAM : 0.004212 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:44:34] Energy consumed for all GPUs : 0.052344 kWh. Total GPU Power : 64.18457773956793 W
[codecarbon INFO @ 09:44:34] Energy consumed for all CPUs : 0.037690 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:44:34] 0.094246 kWh of electricity used since the beginning.


creating index...
index created!
Test:  [ 0/50]  eta: 0:00:18  model_time: 0.1749 (0.1749)  evaluator_time: 0.0050 (0.0050)  time: 0.3766  data: 0.1952  max mem: 11997
Test:  [49/50]  eta: 0:00:00  model_time: 0.1020 (0.1066)  evaluator_time: 0.0036 (0.0056)  time: 0.1195  data: 0.0073  max mem: 11997
Test: Total time: 0:00:06 (0.1290 s / it)
Averaged stats: model_time: 0.1020 (0.1066)  evaluator_time: 0.0036 (0.0056)
Accumulating evaluation results...
DONE (t=0.03s).
Accumulating evaluation results...
DONE (t=0.02s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.857
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.972
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.972
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.118
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.769
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | m

2023/10/16 09:44:43 WARNING mlflow.utils.requirements_utils: Found torch version (2.0.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.0.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2023/10/16 09:44:46 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.15.2+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torchvision==0.15.2' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
[codecarbon INFO @ 09:44:49] Energy consumed for RAM : 0.006574 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:44:49] Energy consumed for all GPUs : 0.075917 kWh. Total GPU Power : 46.717480900689864 W
[codecarbon INFO @ 

In [ ]:
torch.save(model2, 'batch8.pth')

In [ ]:
BATCH_SIZE_TRAIN = 4

# redefine the data loader
data_loader = torch.utils.data.DataLoader(
    training_dataset, batch_size=BATCH_SIZE_TRAIN, shuffle=True, num_workers=2,
    collate_fn=utils.collate_fn)

In [ ]:
tracker.start()
train_mlflow(model2, data_loader, data_loader_val, num_epochs=3, hidden_layer=HIDDEN_LAYER,
    batch_size_train=BATCH_SIZE_TRAIN, name="batch4", device=device)
tracker.stop()

Epoch: [0]  [ 0/70]  eta: 0:02:12  lr: 0.000077  loss: 0.1298 (0.1298)  loss_classifier: 0.0104 (0.0104)  loss_box_reg: 0.0122 (0.0122)  loss_mask: 0.1049 (0.1049)  loss_objectness: 0.0013 (0.0013)  loss_rpn_box_reg: 0.0012 (0.0012)  time: 1.8944  data: 0.3890  max mem: 11997


[codecarbon INFO @ 09:45:34] Energy consumed for RAM : 0.004291 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:45:34] Energy consumed for RAM : 0.006633 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:45:34] Energy consumed for all GPUs : 0.076398 kWh. Total GPU Power : 53.00505981038547 W
[codecarbon INFO @ 09:45:34] Energy consumed for all CPUs : 0.059351 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:45:34] 0.142383 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:45:34] Energy consumed for all GPUs : 0.053019 kWh. Total GPU Power : 53.09956168977472 W
[codecarbon INFO @ 09:45:34] Energy consumed for all CPUs : 0.038402 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:45:34] 0.095713 kWh of electricity used since the beginning.


Epoch: [0]  [10/70]  eta: 0:01:19  lr: 0.000801  loss: 0.1096 (0.1094)  loss_classifier: 0.0098 (0.0093)  loss_box_reg: 0.0078 (0.0079)  loss_mask: 0.0911 (0.0911)  loss_objectness: 0.0001 (0.0002)  loss_rpn_box_reg: 0.0010 (0.0010)  time: 1.3317  data: 0.0538  max mem: 11997


[codecarbon INFO @ 09:45:49] Energy consumed for RAM : 0.004311 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:45:49] Energy consumed for RAM : 0.006653 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:45:49] Energy consumed for all GPUs : 0.053307 kWh. Total GPU Power : 69.15161753745753 W
[codecarbon INFO @ 09:45:49] Energy consumed for all CPUs : 0.038579 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:45:49] 0.096198 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:45:49] Energy consumed for all GPUs : 0.076686 kWh. Total GPU Power : 69.09974522609016 W
[codecarbon INFO @ 09:45:49] Energy consumed for all CPUs : 0.059528 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:45:49] 0.142868 kWh of electricity used since the beginning.


Epoch: [0]  [20/70]  eta: 0:01:04  lr: 0.001525  loss: 0.1038 (0.1052)  loss_classifier: 0.0076 (0.0079)  loss_box_reg: 0.0062 (0.0069)  loss_mask: 0.0883 (0.0895)  loss_objectness: 0.0001 (0.0001)  loss_rpn_box_reg: 0.0007 (0.0008)  time: 1.2587  data: 0.0200  max mem: 11997


...

[codecarbon INFO @ 09:52:21] Energy consumed for RAM : 0.004828 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:52:21] Energy consumed for all GPUs : 0.060265 kWh. Total GPU Power : 45.40912642345941 W
[codecarbon INFO @ 09:52:21] Energy consumed for all CPUs : 0.043202 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:52:21] 0.108294 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:52:21] Energy consumed for RAM : 0.007170 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:52:21] Energy consumed for all GPUs : 0.083644 kWh. Total GPU Power : 44.90867379936529 W
[codecarbon INFO @ 09:52:21] Energy consumed for all CPUs : 0.064151 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:52:21] 0.154964 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:52:36] Energy consumed for RAM : 0.004847 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:52:36] Energy consumed for RAM : 0.007189 kWh. RAM Power : 4.7543792724609375 W
[cod

Test:  [69/70]  eta: 0:00:00  model_time: 0.5193 (0.5140)  evaluator_time: 0.0209 (0.0239)  time: 0.5702  data: 0.0185  max mem: 11997
Test: Total time: 0:00:40 (0.5746 s / it)
Averaged stats: model_time: 0.5193 (0.5140)  evaluator_time: 0.0209 (0.0239)
Accumulating evaluation results...
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.01s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.256
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.257
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.257
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.255
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.256
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.088
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=

[codecarbon INFO @ 09:53:06] Energy consumed for RAM : 0.004887 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:53:06] Energy consumed for all GPUs : 0.061073 kWh. Total GPU Power : 57.714467124446486 W
[codecarbon INFO @ 09:53:06] Energy consumed for all CPUs : 0.043733 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:53:06] 0.109692 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:53:06] Energy consumed for RAM : 0.007229 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:53:06] Energy consumed for all GPUs : 0.084451 kWh. Total GPU Power : 57.30747129534052 W
[codecarbon INFO @ 09:53:06] Energy consumed for all CPUs : 0.064683 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:53:06] 0.156363 kWh of electricity used since the beginning.


Test:  [49/50]  eta: 0:00:00  model_time: 0.0993 (0.1060)  evaluator_time: 0.0024 (0.0054)  time: 0.1107  data: 0.0045  max mem: 11997
Test: Total time: 0:00:06 (0.1298 s / it)
Averaged stats: model_time: 0.0993 (0.1060)  evaluator_time: 0.0024 (0.0054)
Accumulating evaluation results...
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.01s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.846
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.969
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.969
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.135
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.756
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.871
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.415
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 

2023/10/16 09:53:12 WARNING mlflow.utils.requirements_utils: Found torch version (2.0.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.0.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2023/10/16 09:53:15 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.15.2+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torchvision==0.15.2' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
[codecarbon INFO @ 09:53:21] Energy consumed for RAM : 0.004907 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 09:53:21] Energy consumed for all GPUs : 0.061243 kWh. Total GPU Power : 40.87282423821287 W
[codecarbon INFO @ 0

In [ ]:
torch.save(model2, 'batch4.pth')

## Test the final model

In [ ]:
# load best model
final_model = torch.load('baseline.pth')

In [ ]:
tracker.start()
evaluation_mflow('test', data_loader_test, final_model, device)

creating index...
index created!
Test:  [ 0/50]  eta: 0:00:27  model_time: 0.1602 (0.1602)  evaluator_time: 0.0120 (0.0120)  time: 0.5584  data: 0.3845  max mem: 11997


[codecarbon INFO @ 10:07:51] Energy consumed for RAM : 0.008397 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 10:07:51] Energy consumed for all GPUs : 0.092336 kWh. Total GPU Power : 38.10320701031928 W
[codecarbon INFO @ 10:07:51] Energy consumed for all CPUs : 0.075129 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 10:07:51] 0.175862 kWh of electricity used since the beginning.


Test:  [49/50]  eta: 0:00:00  model_time: 0.0993 (0.1046)  evaluator_time: 0.0025 (0.0056)  time: 0.1102  data: 0.0043  max mem: 11997
Test: Total time: 0:00:06 (0.1271 s / it)
Averaged stats: model_time: 0.0993 (0.1046)  evaluator_time: 0.0025 (0.0056)
Accumulating evaluation results...
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.01s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.820
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.989
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.934
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.650
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.663
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.838
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.399
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 

## Stop Emissions Tracker

In [ ]:
tracker.stop()

[codecarbon INFO @ 10:07:21] Energy consumed for RAM : 0.006015 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 10:07:21] Energy consumed for all GPUs : 0.068670 kWh. Total GPU Power : 30.142543061347514 W
[codecarbon INFO @ 10:07:21] Energy consumed for all CPUs : 0.053822 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 10:07:21] 0.128507 kWh of electricity used since the beginning.


0.05816478144549445

[codecarbon INFO @ 10:07:21] Energy consumed for RAM : 0.008358 kWh. RAM Power : 4.7543792724609375 W
[codecarbon INFO @ 10:07:21] Energy consumed for all GPUs : 0.092051 kWh. Total GPU Power : 30.117728331570255 W
[codecarbon INFO @ 10:07:21] Energy consumed for all CPUs : 0.074775 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 10:07:21] 0.175183 kWh of electricity used since the beginning.


With this notebook, we have successfully trained various models with different hyperparameters to determine which was the best model for our experiment, all while tracking the usage of MLFlow for different evaluation metrics and code emissions.